# Scripts to Test Warehouse Summary Data
## Peter Stoermer

In [1]:
# Libraries
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Files Needed For Script to Run
1. 7-SI & 13-SI
2. 7-SV & 13-SV
3. 7-ON & 13-ON
4. 7-SI-INV & 13-SI-INV
5. 7-SV-INV & 13-SV-INV
6. 7-ON-INV & 13-ON-INV
7. Inventory Report for Audit

#### Methodology
The AIMS sales and inventory data gets appended together to make one large data set for each warehouse. Pivot tables are done on that new large dataset to find *sales* and *inventory* figures by date (instruction how to convert invoice date to month is shown below). Once we isolate month I ran pivot tables on the files using month as the index with the different values we needed. Once those isolated dataframes are ran I combinded them and took totals both across rows and down columns. From there I apply the same logic to combine the inventory data from the three warehouses. I then find the total space and units from that data for the summary of the data when it is pulled (thus the date_time cell). To find the amount and space of product going out, I merged the inventory audit data and the combinded sales data to form a master dataframe (this part was difficult because there is not one unique identifier for the products, so I had to make one using both the product stlye and color to find how much space "CBM" is used for each product). From there I ran another pivot table for both CBM and CBF by month. 

Once all of the dataframes are together, I wrote a function that could write all of the dataframes to a single sheet in an excel file so it can be shared, worked on and viewed easily.

### SI Sales Data
The first step is to get the information for both 7 & 13. I then combined them using an *.append()* method because both flat-files have the same structure. Once all of the data is collected I ouput the pivot table looking at the sales data. These are the major steps needed.
1. Gather AIMS Data
2. Using the #-WH file make a new "month" column
 - Done by creating a Datetime object using the "invdate" column
3. Use the pivot tables I made using the Pandas python library

In [3]:
si_7_raw = pd.read_excel(r'C:\Users\Peter Stoermer\OneDrive - Enchante Living\Documents\Joint_Project_Test\7-SI.xls')
si_13_raw = pd.read_excel(r'C:\Users\Peter Stoermer\OneDrive - Enchante Living\Documents\Joint_Project_Test\13-SI.xls')
si_data = si_7_raw.append(si_13_raw)
si_data['month'] = pd.DatetimeIndex(si_data['invdate']).month
si_data.head()

,division,linecode,style,color,desc,catcode,upcno,shptot,price,amount,...,order,entered,edifile,note1,ststate,stzip,piktkt,piktktdate,ediitem,month
0,D,EC,EB197,BNI,SglHdlBatroom Faucet,nan,92903951541,1,81.89,81.89,...,816949,18-Apr-22,NaN,NaN,ON,K2E 1C5,812467,18-Apr-22,EB197,4
1,D,EC,EB197,BNI,SglHdlBatroom Faucet,nan,92903951541,1,81.89,81.89,...,817355,03-May-22,NaN,NaN,ON,K2E 1C5,812835,03-May-22,EB197,5
2,D,EC,EB197,BNI,SglHdlBatroom Faucet,nan,92903951541,3,81.89,245.67,...,817476,09-May-22,NaN,NaN,ON,K2E 1C5,812955,09-May-22,EB197,5
3,D,EC,EB197,BNI,SglHdlBatroom Faucet,nan,92903951541,1,81.89,81.89,...,817573,11-May-22,NaN,NaN,ON,K2E 1C5,813044,11-May-22,EB197,5
4,D,EC,EB197,BNI,SglHdlBatroom Faucet,nan,92903951541,2,81.89,163.78,...,819630,22-Jul-22,NaN,NaN,ON,K2E 1C5,815063,22-Jul-22,EB197,7


In [4]:
si_sales_data = si_data.pivot_table(index=['month'], values=['amount'], aggfunc='sum', margins=True)
si_sales_data = si_sales_data.rename(columns={'amount':'si_sales'})

si_sales_data

,si_sales
month,
1,87044.98
2,116577.85
3,185044.41
4,54819.43
5,131119.51
6,99001.17
7,94188.89
8,103340.02
9,23641.62


### SI Inventory Data

In [5]:
si_inventory_data = si_data.pivot_table(index=['month'], values=['shptot'], aggfunc='sum', margins=True)
si_inventory_data = si_inventory_data.rename(columns={'shptot':'si_inv'})

si_inventory_data

,si_inv
month,
1,3014
2,3508
3,5468
4,1039
5,1873
6,1149
7,1892
8,2172
9,481


### ON Sales Data

In [6]:
on_7_raw = pd.read_excel(r'C:\Users\Peter Stoermer\OneDrive - Enchante Living\Documents\Joint_Project_Test\7-ON.xls')
on_13_raw = pd.read_excel(r'C:\Users\Peter Stoermer\OneDrive - Enchante Living\Documents\Joint_Project_Test\13-ON.xls')
on_data = on_7_raw.append(on_13_raw)
on_data['month'] = pd.DatetimeIndex(on_data['invdate']).month
on_data.head()

,division,linecode,style,color,desc,catcode,upcno,shptot,price,amount,...,order,entered,edifile,note1,ststate,stzip,piktkt,piktktdate,ediitem,month
0,A,IN,AC006Q,CLR,QTIP HOLDER,nan,92903935657.00,720,1.50,1080.00,...,818131,19-May-22,rs0525o.2ag,VENDOR# 201650,CA,92551,813560.00,25-May-22,400247961551.00,6
1,A,IN,AC006Q,CLR,QTIP HOLDER,nan,92903935657.00,360,1.50,540.00,...,818442,26-May-22,rs0531o.2ar,VENDOR# 201650,CA,92551,813924.00,01-Jun-22,400247961551.00,7
2,A,IN,AC006Q,CLR,QTIP HOLDER,nan,92903935657.00,900,1.50,1350.00,...,819295,08-Jul-22,bu0708o.2ab,NaN,CA,92408,814751.00,08-Jul-22,28512457.00,7
3,A,IN,AC008Q,CLR,2 SECTION ROUND ORGA,nan,92903935633.00,896,1.75,1568.00,...,819296,08-Jul-22,bu0708o.2ab,NaN,CA,92408,814752.00,08-Jul-22,28512499.00,7
4,A,IN,AC009Q,CLR,3 SECTION RECTANGLE,nan,92903935718.00,600,2.60,1560.00,...,818131,19-May-22,rs0525o.2ag,VENDOR# 201650,CA,92551,813560.00,25-May-22,400247961568.00,6


In [7]:

on_sales_data = on_data.pivot_table(index=['month'], values=['amount'], aggfunc='sum', margins=True)
on_sales_data = on_sales_data.rename(columns={'amount':'on_sales'})

on_sales_data

,on_sales
month,
1,543840.26
2,662820.98
3,376896.87
4,366437.41
5,107552.70
6,423283.87
7,918518.20
8,2807595.56
9,341555.86


### ON Inventory Data

In [8]:
on_inventory_data = on_data.pivot_table(index=['month'], values=['shptot'], aggfunc='sum', margins=True)
on_inventory_data = on_inventory_data.rename(columns={'shptot':'on_inv'})

on_inventory_data

,on_inv
month,
1,157621
2,153773
3,69874
4,104702
5,28082
6,87036
7,139083
8,427753
9,71018


### SV Sales Data

In [9]:
sv_7_raw = pd.read_excel(r'C:\Users\Peter Stoermer\OneDrive - Enchante Living\Documents\Joint_Project_Test\7-SV.xls')
sv_13_raw = pd.read_excel(r'C:\Users\Peter Stoermer\OneDrive - Enchante Living\Documents\Joint_Project_Test\13-SV.xls')
sv_data = sv_7_raw.append(sv_13_raw)
sv_data['month'] = pd.DatetimeIndex(sv_data['invdate']).month
sv_data.head()

,division,linecode,style,color,desc,catcode,upcno,shptot,price,amount,...,order,entered,edifile,note1,ststate,stzip,piktkt,piktktdate,ediitem,month
0,A,IN,LS015I,LBL,PEVA Shower Curtain,nan,92903950377,1536,2.45,3763.20,...,813367,08-Nov-21,rs1202o.1aa,NO SKU NEEDED/ NO UPC'S NEEDED,CA,92572,809618.00,22-Dec-21,400241400803.00,1
1,A,IN,LS015I,LBL,PEVA Shower Curtain,nan,92903950377,3072,2.50,7680.00,...,814806,25-Jan-22,bu0125o.2aa,NaN,NJ,08010,810458.00,25-Jan-22,31729703.00,2
2,A,IN,LS015I,LGR,PEVA Shower Curtain,nan,92903950384,1536,2.45,3763.20,...,813367,08-Nov-21,rs1202o.1aa,NO SKU NEEDED/ NO UPC'S NEEDED,CA,92572,809618.00,22-Dec-21,400241400797.00,1
3,A,IN,LS015I,LGR,PEVA Shower Curtain,nan,92903950384,3072,2.50,7680.00,...,814805,25-Jan-22,bu0125o.2aa,NaN,NJ,08010,810457.00,25-Jan-22,31729680.00,2
4,A,IN,LS015S,LBL,PEVA Shower Curtain,nan,92903950407,600,2.20,1320.00,...,812053,02-Nov-21,NaN,NO UPC'S NEEDED,WV,26508-8853,809614.00,22-Dec-21,nan,1


In [10]:
sv7_p = pd.read_excel(r'C:\Users\Peter Stoermer\OneDrive - Enchante Living\Documents\Joint_Project_Test\Test(7-SV).xlsx')
sv13_p = pd.read_excel(r'C:\Users\Peter Stoermer\OneDrive - Enchante Living\Documents\Joint_Project_Test\Test(13-SV).xlsx')
sv_data = sv7_p.append(sv13_p, sort=True)
sv_sales_data = sv_data.pivot_table(index=['month'], values=['amount'], aggfunc='sum', margins=True)
sv_sales_data = sv_sales_data.rename(columns={'amount':'sv_sales'})

sv_sales_data

,sv_sales
month,
1,118941.78
2,184252.45
3,152082.98
4,178057.89
5,44729.91
6,18858.24
7,61741.45
8,24628.34
9,32604.79


### SV Inventory Data

In [11]:
sv_inventory_data = sv_data.pivot_table(index=['month'], values=['shptot'], aggfunc='sum', margins=True)
sv_inventory_data = sv_inventory_data.rename(columns={'shptot':'sv_inv'})

sv_inventory_data

,sv_inv
month,
1,33647
2,54684
3,14572
4,32796
5,2662
6,383
7,987
8,420
9,884


## Merged Dataframes (Sales)

In [12]:
all_sales = pd.concat([sv_sales_data, si_sales_data, on_sales_data], axis=1, join='inner')
all_sales = all_sales.eval('All = si_sales + on_sales + sv_sales')
print(all_sales)

       sv_sales  si_sales   on_sales        All
month                                          
1     118941.78  87044.98  543840.26  749827.02
2     184252.45 116577.85  662820.98  963651.28
3     152082.98 185044.41  376896.87  714024.26
4     178057.89  54819.43  366437.41  599314.73
5      44729.91 131119.51  107552.70  283402.12
6      18858.24  99001.17  423283.87  541143.28
7      61741.45  94188.89  918518.20 1074448.54
8      24628.34 103340.02 2807595.56 2935563.92
9      32604.79  23641.62  341555.86  397802.27
All   815897.83 894777.88 6548501.71 8259177.42


## Merged DataFrames (Inventory)

In [13]:
all_inv = pd.concat([sv_inventory_data, si_inventory_data, on_inventory_data], axis=1, join='inner')
all_inv = all_inv.eval('All = sv_inv + si_inv + on_inv')
print(all_inv)

       sv_inv  si_inv   on_inv      All
month                                  
1       33647    3014   157621   194282
2       54684    3508   153773   211965
3       14572    5468    69874    89914
4       32796    1039   104702   138537
5        2662    1873    28082    32617
6         383    1149    87036    88568
7         987    1892   139083   141962
8         420    2172   427753   430345
9         884     481    71018    72383
All    141035   20596  1238942  1400573


## Append 13 + 7 Files for Inventory Data (Date by Warehouse Data)
We are not going to be able to run all of the historic data but we will be able to append both of the charts with cubic (using on_cubic, si_cubic, sv_cubic). Number of units is (on, si, sv). This code uses very similar path structure because all of the data pulled from AIMS is in the same *Raw Files* Folder.

In [14]:
#### Number of Units ON ####
on_13_inv = pd.read_excel(r'C:\Users\Peter Stoermer\OneDrive - Enchante Living\Documents\Joint_Project_Test\13-ON-Inv.xls')
on_7_inv = pd.read_excel(r'C:\Users\Peter Stoermer\OneDrive - Enchante Living\Documents\Joint_Project_Test\7-ON-Inv.xls')
on_units = on_13_inv.append(on_7_inv)
on_units_sum = on_units['on'].sum()
print(f' Total sum of units in ON warehouse: {on_units_sum}')

 Total sum of units in ON warehouse: 1029987


In [15]:
#### Number of Units SI ####
si_13_inv = pd.read_excel(r'C:\Users\Peter Stoermer\OneDrive - Enchante Living\Documents\Joint_Project_Test\13-SI-Inv.xls')
si_7_inv = pd.read_excel(r'C:\Users\Peter Stoermer\OneDrive - Enchante Living\Documents\Joint_Project_Test\7-SI-Inv.xls')
si_units = si_13_inv.append(si_7_inv)
si_units_sum = si_units['si'].sum()
print(f' Total sum of units in SI warehouse: {si_units_sum}')

 Total sum of units in SI warehouse: 48015


In [16]:
#### Number of Units SV ####
sv_13_inv = pd.read_excel(r'C:\Users\Peter Stoermer\OneDrive - Enchante Living\Documents\Joint_Project_Test\13-SV-Inv.xls')
sv_7_inv = pd.read_excel(r'C:\Users\Peter Stoermer\OneDrive - Enchante Living\Documents\Joint_Project_Test\7-SV-Inv.xls')
sv_units = sv_13_inv.append(sv_7_inv)
sv_units_sum = sv_units['sv'].sum()
print(f' Total sum of units in SV warehouse: {sv_units_sum}')

 Total sum of units in SV warehouse: 28508


In [17]:
#### Final Output for Section ####
units_series = [[pd.Timestamp.today(), "Units", on_units_sum, si_units_sum, sv_units_sum]]
units = pd.DataFrame(units_series)
#pd.pivot(units, values=['on_units_sum', 'sv_units_sum', 'si_units_sum'])
#units.reset_index(drop=True)
units.columns = ['Date', 'Units', 'ON', 'SI', 'SV']
units["All"] = units.sum(axis=1)
units

,Date,Units,ON,SI,SV,All
0,2022-09-21 16:48:17.354862,Units,1029987,48015,28508,1106510


## Date by Warehouse Space

In [18]:
#### Space of Units ON ####
on_space = on_units
on_space_sum = round(on_space['on_cubic'].sum(), 2)
print(f' Total sum of units in ON warehouse: {on_space_sum}')

 Total sum of units in ON warehouse: 217848.25


In [19]:
#### Space of Units SI ####
si_space = si_units
si_space_sum = round(si_space['si_cubic'].sum(), 2)
print(f' Total sum of units in SI warehouse: {si_space_sum}')

 Total sum of units in SI warehouse: 33471.65


In [20]:
#### Space of Units SV ####
sv_space = sv_units
sv_space_sum = round(sv_space['sv_cubic'].sum(), 2)
print(f' Total sum of units in SV warehouse: {sv_space_sum}')

 Total sum of units in SV warehouse: 29051.24


In [21]:
#### Final Output for Section ####
space_series = [[pd.Timestamp.today(), "Cubic", on_space_sum, si_space_sum, sv_space_sum]]
space = pd.DataFrame(space_series)
#pd.pivot(units, values=['on_units_sum', 'sv_units_sum', 'si_units_sum'])
#units.reset_index(drop=True)
space.columns = ['Date', 'Cubic', 'ON', 'SI', 'SV']
space["All"] = space.sum(axis=1)
space

,Date,Cubic,ON,SI,SV,All
0,2022-09-21 16:48:19.603901,Cubic,217848.25,33471.65,29051.24,280371.14


## CBF & CBM
We are using the inventory data for this section (for SI (7-SI-Inv & 13-SI-Inv)). They are already combined so all we have to do is isolate the columns for the appropriate information. The formula for cubic ft is $CBF = cubic \, ft / master * (on/si/sv)$. To get meters the formula is $CBM=CBF/35.3$ 

In [22]:
# NOTES: I have to change the first row to lower case in order to join both of the df's together to get month and CBM data
inv_report = pd.read_excel(r'C:\Users\Peter Stoermer\OneDrive - Enchante Living\Documents\Joint_Project_Test\Copy of Inventory Report - For Audit - September 12, 2022.xlsx', header=2)
inv_report.head()

,Division,Category,ID,Account,AIMS Data,AIMS Division,style,color,Type,Status,...,Less Than 3 Months,Over 3 Months,Over 6 Months,Over 9 Months,Over 12 Months,Less Than 3 Months Value ELC,Over 3 Months Value ELC,Over 6 Months Value ELC,Over 9 Months Value ELC,Over 12 Months Value ELC
0,Bathroom Acc,Bathroom Acc,30900121008,Alejandra,7,D,30900121,008,In-Line,Active,...,0,0,0,0,192,0.00,0.00,0.00,0.00,456.96
1,Bathroom Acc,Bathroom Acc,30900131008,Alejandra,7,D,30900131,008,In-Line,Active,...,0,0,0,0,360,0.00,0.00,0.00,0.00,583.20
2,Bathroom Acc,Bathroom Acc,30900141008,Alejandra,7,D,30900141,008,In-Line,Active,...,0,0,0,0,336,0.00,0.00,0.00,0.00,635.04
3,Bathroom Acc,Bathroom Acc,30900151020,Alejandra,7,D,30900151,020,In-Line,Active,...,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00
4,Bathroom Acc,Bathroom Acc,30900191029,Alejandra,7,D,30900191,029,In-Line,Active,...,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00


#### SI CBM & CBF

In [26]:
si_data = si_data.merge(inv_report, on=['style', 'color'], suffixes=('_left', '_right'))


In [27]:
si_data['total_cbm'] = si_data['CRTN CBM_left']/si_data["Casepack_left"]*si_data["shptot"]
cbm_si = si_data.pivot_table(index=['month'], values=['total_cbm'], aggfunc='sum', margins=True)
cbm_si = cbm_si.rename(columns={'total_cbm':'SI_CBM'})
print(cbm_si)

       SI_CBM
month        
1       59.93
2       56.67
3       58.31
4       31.69
5       76.21
6       59.72
7       42.98
8       47.75
9        9.59
All    442.85


In [28]:
si_data['total_cbf'] = si_data['total_cbm']*35.3
cbf_si = si_data.pivot_table(index=['month'], values=['total_cbf'], aggfunc='sum', margins=True)
cbf_si = cbf_si.rename(columns={'total_cbf':'SI_CBF'})
print(cbf_si)

        SI_CBF
month         
1      2115.68
2      2000.31
3      2058.34
4      1118.72
5      2690.29
6      2107.99
7      1517.06
8      1685.63
9       338.41
All   15632.43


#### SV CBM & CBF

In [30]:
sv_data = sv_data.merge(inv_report, on=['style', 'color'], suffixes=('_left', '_right'))

In [31]:
sv_data['total_cbm'] = sv_data['CRTN CBM_left']/sv_data["Casepack_left"]*sv_data["shptot"]
cbm_sv = sv_data.pivot_table(index=['month'], values=['total_cbm'], aggfunc='sum', margins=True)
cbm_sv = cbm_sv.rename(columns={'total_cbm':'SV_CBM'})
print(cbm_sv)

       SV_CBM
month        
1      186.64
2      194.92
3       11.14
4       77.01
5       36.26
6        8.03
7       69.54
8       27.14
9       10.21
All    620.90


In [32]:
sv_data['total_cbf'] = sv_data['total_cbm']*35.3
cbf_sv = sv_data.pivot_table(index=['month'], values=['total_cbf'], aggfunc='sum', margins=True)
cbf_sv = cbf_sv.rename(columns={'total_cbf':'SV_CBF'})
print(cbf_sv)

        SV_CBF
month         
1      6588.26
2      6880.70
3       393.38
4      2718.45
5      1279.99
6       283.49
7      2454.72
8       958.14
9       360.48
All   21917.61


#### ON CBM & CBF

In [34]:
on_data = on_data.merge(inv_report, on=['style', 'color'], suffixes=('_left', '_right'))

In [35]:
on_data['total_cbm'] = on_data['CRTN CBM_left']/on_data["Casepack_left"]*on_data["shptot"]
cbm_on = on_data.pivot_table(index=['month'], values=['total_cbm'], aggfunc='sum', margins=True)
cbm_on = cbm_on.rename(columns={'total_cbm':'ON_CBM'})
print(cbm_on)

       ON_CBM
month        
1      445.93
2      478.92
3      286.41
4      270.24
5       59.07
6      222.43
7      839.38
8     2156.53
9      314.67
All   5073.58


In [36]:
on_data['total_cbf'] = on_data['total_cbm']*35.3
cbf_on = on_data.pivot_table(index=['month'], values=['total_cbf'], aggfunc='sum', margins=True)
cbf_on = cbf_on.rename(columns={'total_cbf':'ON_CBF'})
print(cbf_on)

         ON_CBF
month          
1      15741.16
2      16905.77
3      10110.15
4       9539.58
5       2085.06
6       7851.89
7      29630.22
8      76125.65
9      11107.85
All   179097.33


### Combine CBM & CBF Data Frames

In [37]:
### CBM SV, SI, ON

cbm = pd.concat([cbm_sv, cbm_si, cbm_on], axis=1, join='inner')
cbm = cbm.eval('All = SV_CBM + SI_CBM + ON_CBM')
print(cbm)

       SV_CBM  SI_CBM  ON_CBM     All
month                                
1      186.64   59.93  445.93  692.50
2      194.92   56.67  478.92  730.50
3       11.14   58.31  286.41  355.86
4       77.01   31.69  270.24  378.94
5       36.26   76.21   59.07  171.54
6        8.03   59.72  222.43  290.18
7       69.54   42.98  839.38  951.90
8       27.14   47.75 2156.53 2231.43
9       10.21    9.59  314.67  334.47
All    620.90  442.85 5073.58 6137.32


In [38]:
### CBF SV, SI, ON

cbf = pd.concat([cbf_sv, cbf_si, cbf_on], axis=1, join='inner')
cbf = cbf.eval('All =  SV_CBF + SI_CBF + ON_CBF')
print(cbf)

        SV_CBF   SI_CBF    ON_CBF       All
month                                      
1      6588.26  2115.68  15741.16  24445.10
2      6880.70  2000.31  16905.77  25786.79
3       393.38  2058.34  10110.15  12561.86
4      2718.45  1118.72   9539.58  13376.76
5      1279.99  2690.29   2085.06   6055.33
6       283.49  2107.99   7851.89  10243.37
7      2454.72  1517.06  29630.22  33602.00
8       958.14  1685.63  76125.65  78769.43
9       360.48   338.41  11107.85  11806.73
All   21917.61 15632.43 179097.33 216647.36


In [39]:
#### CBM & CBF for SI ####

si_units["cbf"] = si_units["cubic_ft"] / si_units["master_pack"] * si_units["si"]
si_cbf = round(si_units["cbf"].sum(), 2)
print(f"The total CBF for SI is: {si_cbf}")

print()

si_units["cbm"] = si_units["cubic_ft"] / si_units["master_pack"] * si_units["si"] * 35.3
si_cbm = round(si_units['cbm'].sum(), 2)
print(f"The total CBM for SI is: {si_cbm}")

The total CBF for SI is: 13767.79

The total CBM for SI is: 486002.99


In [40]:
#### CBM & CBF for ON ####

sv_units["cbf"] = sv_units["cubic_ft"] / sv_units["master_pack"] * sv_units["sv"]
sv_cbf = round(sv_units["cbf"].sum(), 2)
print(f"The total CBF for SV is: {sv_cbf}")

print()

sv_units["cbm"] = sv_units["cubic_ft"] / sv_units["master_pack"] * sv_units["sv"] / 35.3
sv_cbm = round(sv_units['cbm'].sum(), 2)
print(f"The total CBM for SV is: {sv_cbm}")

The total CBF for SV is: 26321.31

The total CBM for SV is: 745.65


#### Keeping for Record

sales_master = sales.merge(inv_report, on=['style', 'color'], suffixes=('_left', '_right'))
sales_master['CRTN CBF'] = sales_master['CRTN CBM'].div(35.3)
sales_master.head()

#### Keeping for Record

cbm_month = sales_master.pivot_table(index=['month'], values=['CRTN CBM'], aggfunc='sum', margins=True) 
cbm_month

#### Keeping for Record

cbf_month = sales_master.pivot_table(index=['month'], values=['CRTN CBF'], aggfunc='sum', margins=True) 
cbf_month

In [41]:
## Test for SI CBM & CBF

si_data['total_cbm'] = si_data['CRTN CBM_left']/si_data["Casepack_left"]*si_data["shptot"]
cbm_si = si_data.pivot_table(index=['month'], values=['total_cbm'], aggfunc='sum', margins=True)
cbm_si = cbm_si.rename(columns={'total_cbm':'SI_CBM'})
print(cbm_si)

       SI_CBM
month        
1       59.93
2       56.67
3       58.31
4       31.69
5       76.21
6       59.72
7       42.98
8       47.75
9        9.59
All    442.85


### Testing Outputs

To test the outputs I am going to create an open file using a excel python package. It is easy to make each pivot table its own sheet in the excel workbook. I want to challenge myself and look into the documentation and see if there is a way to have it all work on one sheet (eliminating the chance for human error or incorrect reporting). **The below code works, search Final_Test_09_20.xlsx for the full workbook**

In [42]:
   #### function for multiple DataFrames to one "Summary" sheet ####
def multiple_dfs(df_list, sheets, file_name, spaces):
        writer = pd.ExcelWriter(file_name,engine='xlsxwriter')   
        row = 0
        for dataframe in df_list:
            dataframe.to_excel(writer,sheet_name=sheets,startrow=row , startcol=2)   
            row = row + len(dataframe.index) + spaces + 1
        si_data.to_excel(writer, sheet_name='SI Sales')
        sv_data.to_excel(writer, sheet_name='SV Sales')
        on_data.to_excel(writer, sheet_name='ON Sales')
        si_units.to_excel(writer, sheet_name='SI Units')
        sv_units.to_excel(writer, sheet_name='SV Units')
        on_units.to_excel(writer, sheet_name='ON Units')

        writer.save()

    ##### list of dataframes #####
dfs = [units, space, all_sales, all_inv, cbm, cbf]

    
    #### run function ####
multiple_dfs(dfs, 'Summary', 'Test_09_21.xlsx', 1)